<a href="https://colab.research.google.com/github/Bhardwaj-Saurabh/Agents_Design_Pattern/blob/master/MultiAgent_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
# !pip install openai

In [3]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [10]:
from openai import OpenAI

In [15]:
client = OpenAI()

In [27]:
EASSY_SYSTEM_PROMPT = "You are an eassy writer which writes essay in british english."

In [22]:
TRANSLATOR_SYSTEM_PROMPT = "You are an english to hindi translator. You are given the english essay below. \n %s. \n.Translate it to Hindi language"

In [54]:
def agent(system_prompt, user_query):
    response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )

    return response.choices[0].message.content

In [57]:
def multiagent_pipeline(user_query: str):

    state_dict = dict()

    essay = agent(EASSY_SYSTEM_PROMPT, user_query)
    state_dict['english_essay'] = essay

    if state_dict['english_essay']:
        translation = agent(TRANSLATOR_SYSTEM_PROMPT, state_dict['english_essay'])
        state_dict['hindi_essay'] = translation

        if state_dict['hindi_essay']:
            with open('hindi_essay.txt', 'w+') as f:
                f.write(state_dict['hindi_essay'])
        else:
            print('error translating english essay')
    else:
        print('error while generating english essay')

    return state_dict

In [58]:
state_dict = multiagent_pipeline('write an essay about python snake.')

In [59]:
state_dict['english_essay']

'**The Python Snake: A Fascinating Creature of the Serpentidae Family**\n\nThe Python, a member of the Serpentidae family, is a remarkable creature that has captured the fascination of many, both in the wild and in captivity. Known for its impressive size, unique adaptability, and distinct behaviours, the python has become an iconic symbol of the diverse reptilian world. This essay aims to delve into the characteristics, habitat, diet, and conservation status of pythons, shedding light on why they are both revered and misunderstood in various cultures.\n\n**Physical Characteristics and Variations**\n\nPythons are non-venomous constrictors that exhibit considerable variation in size and colour, depending on the species. The most common species include the reticulated python, green python, and ball python, each possessing unique traits. For instance, the reticulated python is one of the longest snakes in the world, stretching up to 10 metres in length. In contrast, the ball python, typic

In [60]:
state_dict['hindi_essay']

'**Python सांप: सर्पेंटिडे परिवार का एक रोचक प्राणी**\n\nPython, सर्पेंटिडे परिवार का एक सदस्य, एक अद्वितीय प्राणी है जिसने जंगली और कैद दोनों में कई लोगों का ध्यान आकर्षित किया है। अपने प्रभावशाली आकार, अद्वितीय अनुकूलनशीलता और विशिष्ट व्यवहार के लिए जाने जाने वाला, पायथन विविध चिड़ियाघर के प्रतीक के रूप में जाना जाता है। यह निबंध पायथनों की विशेषताओं, आवास, आहार और संरक्षण स्थिति में गहराई से जाने का प्रयास करता है, यह स्पष्ट करते हुए कि वे विभिन्न संस्कृतियों में क्यों पूजे और गलत समझे जाते हैं।\n\n**शारीरिक विशेषताएं और भिन्नताएं**\n\nपायथन गैर-विषैला संकुचन करते हैं जो प्रजातियों के आधार पर आकार और रंग में महत्वपूर्ण भिन्नता प्रदर्शित करते हैं। सबसे सामान्य प्रजातियों में रेटिकुलेटेड पायथन, हरे पायथन और बॉल पायथन शामिल हैं, प्रत्येक में विशेष लक्षण होते हैं। उदाहरण के लिए, रेटिकुलेटेड पायथन दुनिया के सबसे लंबे सांपों में से एक है, जो 10 मीटर तक बढ़ सकता है। इसके विपरीत, बॉल पायथन, जो आमतौर पर अफ्रीका के कुछ भागों में पाया जाता है, काफी छोटा होता है, आमतौर पर 1 से 1.5 मीटर के बीच ह

In [39]:
class OpenAIAgentChained:
    def __init__(self, func):
        self.func = func
    def __call__(self, input_text):
        return self.func(input_text)
    # Add __rshift__ method to allow chaining OpenAIAgentChained objects
    def __rshift__(self, other_agent):
        def chained_function(input_text):
            # Call the current chained function first
            intermediate_result = self(input_text)
            # Then call the next agent with the intermediate result
            return other_agent(intermediate_result)
        # Return a new OpenAIAgentChained object with the combined function
        return OpenAIAgentChained(chained_function)

class OpenAIAgent:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
        self.client = OpenAI()

    def __call__(self, input_text):
        import openai
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": self.prompt_template},
                {"role": "user", "content": input_text}
            ]
        )
        return response.choices[0].message.content

    def __rshift__(self, other_agent):
        def chained_function(input_text):
            result = self(input_text)
            return other_agent(result)
        return OpenAIAgentChained(chained_function)


In [40]:
from pathlib import Path

class SaveToFileAgent:
    def __init__(self, filepath: str):
        self.filepath = Path(filepath)

    def __call__(self, content: str) -> str:
        self.filepath.write_text(content, encoding="utf-8")
        return f"✅ Output saved to {self.filepath.resolve()}"

    def __rshift__(self, other_agent):
        def chained_func(user_input: str):
            output = self(user_input)
            return other_agent(output)
        return OpenAIAgentChained(chained_func)

In [41]:
eassy_write = OpenAIAgent(EASSY_SYSTEM_PROMPT)
translator = OpenAIAgent(TRANSLATOR_SYSTEM_PROMPT)
savefileagent = SaveToFileAgent('/content/eassy.txt')


pipeline = eassy_write >> translator >> savefileagent

In [42]:
result = pipeline('write an eassy about earth from its inception to human advancement with proper heading, subheading and pragraphs.')

In [61]:
from IPython.display import Markdown
Markdown(result)

✅ Output saved to /content/eassy.txt